<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Conversion_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Conversion Calculation
The concentration data obtained from the Concentration_calculation.ipynb is converted according to volume of the air sample. The input files are both a file with the experimental data such as volume, temperature and pressure data (info_*_n.csv)as well as the export file from the Concentration_calculation.ipynb script.

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
import glob
from datetime import datetime
import csv

drive.mount('/content/drive', force_remount = True)

sample_nr_low = 183 #needs to be adapted to 1 nr in the series
sample_nr_high = 184 #needs to be adapted to the last nr of the series

QC_suptraction_nr = 180  #needs to be adapted to the quality control file that should be subtracted.

Mounted at /content/drive


The following part calculates the volume of air from the experimental conditions to standard ambient temperature and presure (SATP) which are 298.15 K and 100000 Pa. The imput file is a .csv file, containing a header of two (sample information and desctiption) as well as the data for the sampled volume, pressure and temperature. info_*{n}.csv is used as general name template, where n is the sample number corresponding to the sample number used for the concentration calculation skript (Concentration_calculation.ipynb). All the info files are uploaded to the conversion_input folder, while the resulting file is saved in the conversion_output under the name template V_SATP_{n}.csv

In [2]:
t_SATP = 293.15
p_SATP = 100000

for n in range(sample_nr_low, sample_nr_high):
  files = glob.glob(f'/content/drive/My Drive/conversion_input/info_*{n}.csv')

  if not files:
    print(f'Files matching pattern Sample Nr. {n} was not found.')
    continue

  else:
    for file in files:
      df = pd.read_csv(file, header=1)
      #print(df)
      V = df.iloc[:, 1]
      t = df.iloc[:, 2]
      p = df.iloc[:, 3]
      #print('Volume:',V)
      #print('temp:', t)
      #print('Pa:', p)
      V_SATP = (p * V * t_SATP) / (t * p_SATP)
      #print(V_SATP)
      V_SATP.to_csv(f'/content/drive/My Drive/conversion_output/V_SATP_{n}.csv', header=None, index=False)
      print(n, V_SATP)

183 0    1.180159
dtype: float64


To get the final concentration in air, the amount of phthalate is divided by the volume at SATP. this is done for eac file in the defined sample nr. range. All the files from the quantitation script (Concentration_calculation.ipynb) are located in the quantitation export folder, while the output of the previous section is saved in the output folder of this conversion script. The final file file is saved in the conversion export folder under the name Sample_{n}_phthalate_concentration_{current_date}.csv.

In [3]:
current_date = datetime.today().strftime('%Y-%m-%d')

for n in range(sample_nr_low, sample_nr_high):
    files = glob.glob(f'/content/drive/My Drive/quantitation_export/Sample_{n}_*.csv')

    if not files:
        print(f'Files matching pattern Sample Nr. {n} was not found.')
        continue

    else:
      df = pd.read_csv(files[0], header=1, index_col=False)
      print(df)
      V_SATP = pd.read_csv(f'/content/drive/My Drive/conversion_output/V_SATP_{n}.csv', header=None, index_col=False)
      C = df.iloc[:, 4]
      if QC_suptraction_nr != 0:
        QC_file = glob.glob(f'/content/drive/My Drive/quantitation_export/Sample_{QC_suptraction_nr}_*.csv')
        QC_subtraction_df = pd.read_csv(QC_file[0], header=1, index_col=False)
        print(QC_subtraction_df)
        columns_to_subtract = df.columns[1:]
        C = df[columns_to_subtract].subtract(QC_subtraction_df[columns_to_subtract], axis='columns')

      else:
        continue

      C = C.mul(1000)
      Compound_info = df.iloc[:, 0]
      print(V_SATP)
      C_air = C.div(V_SATP.iloc[0, 0])

      print('Compoundinfo:', Compound_info)
      print('C_airvalue', C_air)

      output_file = f'/content/drive/My Drive/conversion_export/Sample_{n}_phthalate_concentration_{current_date}.csv'
      with open(output_file, 'w', newline='') as csvfile:
          writer = csv.writer(csvfile)
          header_row1 = ['Sample nr.', n, 'Calculation Date', current_date]
          writer.writerow(header_row1)
          header_row2 = ['Phthalat:', 'Concentration [ng/m^3]']
          writer.writerow(header_row2)
          for i in range(len(Compound_info)):
            writer.writerow([Compound_info.iloc[i],C_air.iloc[i, 3]])

                       Analyte  Intercept     Slope     Value  \
0            Dimethylphthalate   0.001358  0.356679  0.001585   
1             Diethylphthalate   0.001323  0.387867  0.039948   
2             Dibutylphthalate   0.002397  0.656251  0.022847   
3         Benzylbutylphthalate  -0.000574  0.273788  0.000583   
4  Bis(2-ethyl-hexyl)phthalate  -0.001668  0.450815  0.021489   
5          Di-n-octylphthalate  -0.006264  0.490070  0.000863   

   Concentration [microg/ml]  
0                   0.001923  
1                   0.016818  
2                   0.017390  
3                  -0.000414  
4                   0.008020  
5                  -0.005841  
                       Analyte  Intercept     Slope     Value  \
0            Dimethylphthalate   0.001358  0.356679  0.000337   
1             Diethylphthalate   0.001323  0.387867  0.025984   
2             Dibutylphthalate   0.002397  0.656251  0.012242   
3         Benzylbutylphthalate  -0.000574  0.273788  0.000715   
4 